In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Função para gerar espirais
def gerar_espiral(pontos, ruido):
    n = pontos // 2

    # Primeira espiral
    theta1 = np.sqrt(np.random.rand(n)) * 4 * np.pi
    x1 = theta1 * np.cos(theta1) + np.random.normal(0, ruido, n)
    y1 = theta1 * np.sin(theta1) + np.random.normal(0, ruido, n)
    label1 = np.zeros(n)

    # Segunda espiral (defasada de pi)
    theta2 = np.sqrt(np.random.rand(n)) * 4 * np.pi
    x2 = theta2 * np.cos(theta2 + np.pi) + np.random.normal(0, ruido, n)
    y2 = theta2 * np.sin(theta2 + np.pi) + np.random.normal(0, ruido, n)
    label2 = np.ones(n)

    # Juntar dados
    X = np.vstack((np.column_stack((x1, y1)), np.column_stack((x2, y2))))
    y = np.hstack((label1, label2))

    return X, y

# Dados com baixa sobreposição
#X, y = gerar_espiral(300, 0.2)

# Gerar com sobreposição intermediária
#X, y = gerar_espiral(300, 0.5)

# Dados com alta sobreposição
X, y = gerar_espiral(300, 0.7)

# Monta DataFrame para evitar problemas de alinhamento
df = pd.DataFrame({
    'X1': X[:, 0],
    'X2': X[:, 1],
    'Classe': pd.Series(y.astype(int)).map({0: 'Classe 0', 1: 'Classe 1'})
})

# Scatter plot usando DataFrame
scatter = sns.scatterplot(
    data=df,
    x='X1',
    y='X2',
    hue='Classe',
    palette='Set1'
)

# Corrigir legenda (remove título)
handles, labels_leg = scatter.get_legend_handles_labels()
plt.legend(handles=handles, labels=labels_leg, frameon=True)

#plt.title("Problema das Espirais")
plt.xlabel('X1')
plt.ylabel('X2')
#plt.legend(title='Classe')
plt.show()

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Valores de C para testar
valores_C = np.linspace(1, 100, 10)

# Loop sobre os valores de C
for C_val in valores_C:
    # Treinar SVM com kernel RBF
    modelo = SVC(C=C_val, kernel='rbf', gamma='scale')
    modelo.fit(X_train, y_train)

    # Previsões
    y_pred = modelo.predict(X_test)

    # Acurácia e matriz de confusão
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print(f"\nC = {C_val}")
    print(f"Acurácia: {acc:.4f}")
    print("Matriz de Confusão:")
    print(cm)

    # Plot da superfície de decisão
    plt.figure(figsize=(8,6))
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500),
                         np.linspace(y_min, y_max, 500))

    Z = modelo.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Monta DataFrame para evitar problemas de alinhamento
    df = pd.DataFrame({
        'X1': X[:, 0],
        'X2': X[:, 1],
        'Classe': pd.Series(y.astype(int)).map({0: 'Classe 0', 1: 'Classe 1'})
    })

    # Plot da superfície de decisão
    plt.contourf(xx, yy, Z, alpha=0.2, cmap='Set1')

    # Scatter plot usando DataFrame
    scatter = sns.scatterplot(
        data=df,
        x='X1',
        y='X2',
        hue='Classe',
        palette='Set1'
    )

    # Corrigir legenda (remove título)
    handles, labels_leg = scatter.get_legend_handles_labels()
    plt.legend(handles=handles, labels=labels_leg, frameon=True)

    # Títulos e rótulos
    #plt.title("Problema das Espirais")
    plt.xlabel("X1")
    plt.ylabel("X2")

    plt.show()